In [1]:
import hopsworks
import pandas as pd

project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://snurran.hops.works/p/16503
Connected. Call `.close()` to terminate connection gracefully.


In [2]:
df = pd.read_csv("../data/Mensa_occupancy_history.csv", sep=";", parse_dates=["Timestamp"], infer_datetime_format=True)
df['id']="Heilbronn"

2024-08-15 13:11:01,206 WARNING: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

2024-08-15 13:11:01,245 WARNING: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [3]:
df = df.rename(columns={'Kapazität': 'capacity', 'Timestamp' : 'ds', 'Auslastung' : 'y'})
df

,ds,y,capacity,id
0,2024-06-25 12:01:00,198.0,NaN,Heilbronn
1,2024-06-25 12:02:00,214.0,NaN,Heilbronn
2,2024-06-25 12:03:00,249.0,NaN,Heilbronn
3,2024-06-25 12:04:00,279.0,NaN,Heilbronn
4,2024-06-25 12:05:00,287.0,NaN,Heilbronn
...,...,...,...,...
50019,2024-08-13 13:18:00,54.0,500.0,Heilbronn
50020,2024-08-13 13:19:00,54.0,500.0,Heilbronn
50021,2024-08-13 13:20:00,53.0,500.0,Heilbronn
50022,2024-08-13 13:21:00,47.0,500.0,Heilbronn


In [4]:
fg = fs.get_or_create_feature_group(name="mensa", 
                                    version=1, 
                                    description="Time-series capacity load measurements for a Mensa restaurant",
                                    primary_key=["id","ds"], 
                                    event_time="ds",
                                   )
fg.insert(df)

Feature Group created successfully, explore it at 
https://snurran.hops.works/p/16503/fs/16451/fg/17427


Uploading Dataframe: 0.00% |          | Rows 0/50024 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: mensa_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://snurran.hops.works/p/16503/jobs/named/mensa_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f983d485d80>, None)

In [5]:
df2= pd.read_csv("../data/Temprature_Heilbronn_history.csv", sep=";", parse_dates=["Timestamp"], infer_datetime_format=True)
df2['id']="Heilbronn"

2024-08-15 13:11:05,595 WARNING: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



In [6]:
fg = fs.get_or_create_feature_group(name="heilbronn_temperature", 
                                    version=1, 
                                    description="Temperature readings at Heibronn",
                                    primary_key=["id","timestamp"], 
                                    event_time="timestamp",
                                   )
fg.insert(df2)

2024-08-15 13:11:05,651 WARNING: FeatureGroupWarning: The ingested dataframe contains upper case letters in feature names: `['Timestamp']`. Feature names are sanitized to lower case in the feature store.

Feature Group created successfully, explore it at 
https://snurran.hops.works/p/16503/fs/16451/fg/17428


Uploading Dataframe: 0.00% |          | Rows 0/50000 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: heilbronn_temperature_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://snurran.hops.works/p/16503/jobs/named/heilbronn_temperature_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f983e50aa40>, None)

In [7]:
df3= pd.read_csv("../data/waste_bin_filling_level_history.csv", sep=";", parse_dates=["Timestamp"], infer_datetime_format=True)
df3['id']="Heilbronn"
df3

2024-08-15 13:11:07,596 WARNING: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



,Timestamp,fillLevel,id
0,2024-06-04 15:01:22,46,Heilbronn
1,2024-06-04 16:01:38,46,Heilbronn
2,2024-06-04 17:01:54,100,Heilbronn
3,2024-06-04 18:02:10,63,Heilbronn
4,2024-06-04 19:02:26,56,Heilbronn
...,...,...,...
1810,2024-08-13 09:01:22,0,Heilbronn
1811,2024-08-13 10:01:38,0,Heilbronn
1812,2024-08-13 11:01:56,0,Heilbronn
1813,2024-08-13 12:00:16,0,Heilbronn


In [8]:
fg = fs.get_or_create_feature_group(name="waste_bin_filling_level", 
                                    version=1, 
                                    description="Waste bin filling levelat Heibronn",
                                    primary_key=["id", "timestamp"], 
                                    event_time="timestamp",
                                   )
fg.insert(df3)

2024-08-15 13:11:07,628 WARNING: FeatureGroupWarning: The ingested dataframe contains upper case letters in feature names: `['Timestamp', 'fillLevel']`. Feature names are sanitized to lower case in the feature store.

Feature Group created successfully, explore it at 
https://snurran.hops.works/p/16503/fs/16451/fg/17429


Uploading Dataframe: 0.00% |          | Rows 0/1815 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: waste_bin_filling_level_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://snurran.hops.works/p/16503/jobs/named/waste_bin_filling_level_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f98f4124520>, None)